In [1]:
from glob import glob
import mne
from scipy.stats import kurtosis, skew
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import welch
from scipy import stats

In [2]:
all_files_path=glob('Data/*.edf')
print(len(all_files_path))

28


In [3]:
all_files_path

['Data\\h01.edf',
 'Data\\h02.edf',
 'Data\\h03.edf',
 'Data\\h04.edf',
 'Data\\h05.edf',
 'Data\\h06.edf',
 'Data\\h07.edf',
 'Data\\h08.edf',
 'Data\\h09.edf',
 'Data\\h10.edf',
 'Data\\h11.edf',
 'Data\\h12.edf',
 'Data\\h13.edf',
 'Data\\h14.edf',
 'Data\\s01.edf',
 'Data\\s02.edf',
 'Data\\s03.edf',
 'Data\\s04.edf',
 'Data\\s05.edf',
 'Data\\s06.edf',
 'Data\\s07.edf',
 'Data\\s08.edf',
 'Data\\s09.edf',
 'Data\\s10.edf',
 'Data\\s11.edf',
 'Data\\s12.edf',
 'Data\\s13.edf',
 'Data\\s14.edf']

In [4]:
healthy_file_path=[i for i in all_files_path if  'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('\\')[1]]

In [5]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    print(f'data shape:{datax.get_data().shape}')
    epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
    epochs=epochs.get_data()
    return epochs #trials,channel,length

In [6]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from c:\Users\Greatest Pleasure\Downloads\EEG PROCESSING\EEG_Classification_schizophrenia\Data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

data shape:(19, 231250)
Not setting metadata
37 matching

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [7]:
data.shape

(37, 19, 6250)

In [8]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [9]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [10]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

28 28


In [11]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

In [12]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
print(data_array.shape,label_array.shape,group_array.shape)

(1142, 19, 6250) (1142,) (1142,)


In [13]:
def compute_psd(eeg_data, frequency_range, fs=250, nperseg=125):
    epochs, channels, _ = eeg_data.shape
    psd = np.zeros((epochs, channels))

    for epoch in range(epochs):
        for channel in range(channels):
            freqs, power_spectrum = welch(eeg_data[epoch, channel], fs=fs, nperseg=nperseg)
            freq_min, freq_max = frequency_range
            indices = np.logical_and(freqs >= freq_min, freqs <= freq_max)
            psd[epoch, channel] = np.mean(power_spectrum[indices])

    return psd


In [14]:
def compute_mean(eeg_data):
    epochs, channels, _ = eeg_data.shape
    mean_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            mean_values[epoch, channel] = np.mean(eeg_data[epoch, channel])
    
    return mean_values

def compute_std(eeg_data):
    epochs, channels, _ = eeg_data.shape
    std_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            std_values[epoch, channel] = np.std(eeg_data[epoch, channel])
    
    return std_values

def compute_kurtosis(eeg_data):
    epochs, channels, _ = eeg_data.shape
    kurtosis_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            kurtosis_values[epoch, channel] = kurtosis(eeg_data[epoch, channel])
    
    return kurtosis_values

def compute_skewness(eeg_data):
    epochs, channels, _ = eeg_data.shape
    skewness_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            skewness_values[epoch, channel] = skew(eeg_data[epoch, channel])
    
    return skewness_values

def compute_mean_square(eeg_data):
    epochs, channels, _ = eeg_data.shape
    mean_square_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            mean_square_values[epoch, channel] = np.mean(eeg_data[epoch, channel] ** 2)
    
    return mean_square_values

def compute_rms(eeg_data):
    epochs, channels, _ = eeg_data.shape
    rms_values = np.zeros((epochs, channels))
    
    for epoch in range(epochs):
        for channel in range(channels):
            rms_values[epoch, channel] = np.sqrt(np.mean(eeg_data[epoch, channel] ** 2))
    
    return rms_values


In [15]:
mean_values = compute_mean(data_array)[..., np.newaxis]
std_values = compute_std(data_array)[..., np.newaxis]
kurtosis_values = compute_kurtosis(data_array)[..., np.newaxis]
skewness_values = compute_skewness(data_array)[..., np.newaxis]
mean_square_values = compute_mean_square(data_array)[..., np.newaxis]
rms_values = compute_rms(data_array)[..., np.newaxis]

In [16]:
theta = compute_psd(data_array, (4, 8))[..., np.newaxis]
alpha = compute_psd(data_array, (9, 12))[..., np.newaxis]
beta = compute_psd(data_array, (13, 30))[..., np.newaxis]

In [17]:
combined_features = np.concatenate(
    (theta, alpha, beta,mean_values, std_values, kurtosis_values, skewness_values, mean_square_values, rms_values), 
    axis=-1)
features = combined_features.reshape(combined_features.shape[0], -1)
features.shape


(1142, 171)

In [18]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
from sklearn.metrics import confusion_matrix,classification_report

In [19]:
gkf=GroupKFold(n_splits=5)
for train_index, val_index in gkf.split(features, label_array, groups=group_array):
    # Split the data
    train_features, train_labels = features[train_index], label_array[train_index]
    val_features, val_labels = features[val_index], label_array[val_index]
    group_array_fold = group_array[train_index]

In [20]:
train_features.shape

(907, 171)

In [21]:
val_features.shape

(235, 171)

In [22]:
clf = LogisticRegression()
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', clf)])

params_grid = {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
gkf = GroupKFold(n_splits=5)

model = GridSearchCV(pipe, params_grid, cv=gkf)
model.fit(train_features, train_labels,groups=group_array_fold)

C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

In [23]:
final_model = model.best_estimator_
history1=final_model.fit(train_features, train_labels)
y_pred = final_model.predict(val_features)

C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
print(classification_report(val_labels,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.87       110
           1       0.93      0.80      0.86       125

    accuracy                           0.86       235
   macro avg       0.87      0.87      0.86       235
weighted avg       0.87      0.86      0.86       235



In [25]:
print("Training set score for :LogisticRegression %f" % final_model.score(train_features ,train_labels))
print("Testing set score for LogisticRegression: %f" % final_model.score(val_features ,val_labels))

Training set score for :LogisticRegression 0.961411
Testing set score for LogisticRegression: 0.863830


In [26]:
clf = DecisionTreeClassifier()
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', clf)])

params_grid_dt = {'classifier__max_depth': [2, 3, 5, 7, 10, 15, 20], 'classifier__criterion': ['gini', 'entropy']}
model = GridSearchCV(pipe, params_grid_dt, cv=gkf)
model.fit(train_features, train_labels, groups=group_array_fold)


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        DecisionTreeClassifier())]),
             param_grid={'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': [2, 3, 5, 7, 10, 15, 20]})

In [27]:
final_model = model.best_estimator_
history1=final_model.fit(train_features, train_labels)
y_pred = final_model.predict(val_features)

In [28]:
print(classification_report(val_labels,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.88      0.72       110
           1       0.83      0.50      0.62       125

    accuracy                           0.68       235
   macro avg       0.72      0.69      0.67       235
weighted avg       0.72      0.68      0.67       235



In [29]:
print("Training set score for Decision Tree: %f" % model.score(train_features , train_labels))
print("Testing  set score for Decision Tree: %f" % model.score(val_features ,val_labels ))

Training set score for Decision Tree: 1.000000
Testing  set score for Decision Tree: 0.676596


In [30]:
clf = SVC()
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])
params_grid_svm = {'classifier__C': [0.01, 0.1, 1, 10], 'classifier__kernel': ['linear', 'rbf']}
model = GridSearchCV(pipe, params_grid_svm, cv=gkf)
model.fit(train_features, train_labels, groups=group_array_fold)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10],
                         'classifier__kernel': ['linear', 'rbf']})

In [31]:
final_model = model.best_estimator_
history1=final_model.fit(train_features, train_labels)
y_pred = final_model.predict(val_features)

In [32]:
print(classification_report(val_labels,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       110
           1       0.53      1.00      0.69       125

    accuracy                           0.53       235
   macro avg       0.27      0.50      0.35       235
weighted avg       0.28      0.53      0.37       235



C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [33]:
print("Training set score for SVM: %f" % model.score(train_features , train_labels))
print("Testing  set score for SVM: %f" % model.score(val_features ,val_labels ))

Training set score for SVM: 0.552370
Testing  set score for SVM: 0.531915


In [34]:
clf = MLPClassifier(max_iter=500)
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', clf)])

params_grid_mlp = {
    'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 
    'classifier__alpha': [0.0001, 0.001, 0.01]
}
model = GridSearchCV(pipe, params_grid_mlp, cv=gkf)
model.fit(train_features, train_labels, groups=group_array_fold)

C:\Users\Greatest Pleasure\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        MLPClassifier(max_iter=500))]),
             param_grid={'classifier__alpha': [0.0001, 0.001, 0.01],
                         'classifier__hidden_layer_sizes': [(50,), (100,),
                                                            (50, 50),
                                                            (100, 50)]})

In [35]:
final_model_mlp = model.best_estimator_
final_model_mlp.fit(train_features, train_labels)
y_pred = final_model_mlp.predict(val_features)

In [36]:
print(classification_report(val_labels,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       110
           1       0.91      0.91      0.91       125

    accuracy                           0.91       235
   macro avg       0.91      0.91      0.91       235
weighted avg       0.91      0.91      0.91       235



In [37]:
print("Training set score for MLP: %f" % model.score(train_features , train_labels))
print("Testing  set score for MLP: %f" % model.score(val_features ,val_labels ))

Training set score for MLP: 1.000000
Testing  set score for MLP: 0.906383


In [38]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Define GroupKFold cross-validator
gkf = GroupKFold(n_splits=5)

# Define pipelines, parameter grids, and models for each classifier
models = {
    'LogisticRegression': {
        'pipeline': Pipeline([('scaler', StandardScaler()), ('classifier', LogisticRegression())]),
        'param_grid': {'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}
    },
    'DecisionTree': {
        'pipeline': Pipeline([('scaler', StandardScaler()), ('classifier', DecisionTreeClassifier())]),
        'param_grid': {'classifier__max_depth': [2, 3, 5, 7, 10, 15, 20], 'classifier__criterion': ['gini', 'entropy']}
    },
    'MLP': {
        'pipeline': Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=500))]),
        'param_grid': {'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)], 'classifier__alpha': [0.0001, 0.001, 0.01]}
    },
    'SVM': {
        'pipeline': Pipeline([('scaler', StandardScaler()), ('classifier', SVC())]),
        'param_grid': {'classifier__C': [0.01, 0.1, 1, 10], 'classifier__kernel': ['linear', 'rbf']}
    }
}

# Perform training and validation for each model
for model_name, model_info in models.items():
    print(f"\nTraining {model_name} model...")
    pipe = model_info['pipeline']
    params_grid = model_info['param_grid']
    
    # Grid search with GroupKFold cross-validation
    model = GridSearchCV(pipe, params_grid, cv=gkf)
    model.fit(train_features, train_labels, groups=group_array_fold)
    
    # Best model for the current classifier
    final_model = model.best_estimator_
    history = final_model.fit(train_features, train_labels)
    y_pred = final_model.predict(val_features)
    
    # Evaluate model
    accuracy = accuracy_score(val_labels, y_pred)
    print(f"Best parameters for {model_name}: {model.best_params_}")
    print(f"{model_name} Accuracy: {accuracy:.4f}")
'''

'from sklearn.linear_model import LogisticRegression\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.neural_network import MLPClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.model_selection import GroupKFold, GridSearchCV\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.metrics import accuracy_score\n\n# Define GroupKFold cross-validator\ngkf = GroupKFold(n_splits=5)\n\n# Define pipelines, parameter grids, and models for each classifier\nmodels = {\n    \'LogisticRegression\': {\n        \'pipeline\': Pipeline([(\'scaler\', StandardScaler()), (\'classifier\', LogisticRegression())]),\n        \'param_grid\': {\'classifier__C\': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 8, 10, 12, 15]}\n    },\n    \'DecisionTree\': {\n        \'pipeline\': Pipeline([(\'scaler\', StandardScaler()), (\'classifier\', DecisionTreeClassifier())]),\n        \'param_grid\': {\'classifier__max_depth\': [2, 3, 5, 7, 10, 15, 20], \'classifier